In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import re
import csv
import json
import time
import numpy as np
import pandas as pd
from pprint import pprint

from langchain import hub
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
UPSTAGE_API_KEY = os.environ.get('UPSTAGE_API_KEY')
LANGCHAIN_API_KEY = os.environ.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = 'matching_model_demo' # 프로젝트명 수정
LANGCHAIN_PROJECT = os.environ.get('LANGCHAIN_PROJECT')

print(f'LangSmith Project: {LANGCHAIN_PROJECT}')

LangSmith Project: matching_model_demo


# 데이터 구성

> 전처리, csv to jsonl

In [3]:
# 데이터 로드

text = pd.read_excel('../data/비식별된 해외기업별 영문 텍스트데이터.xlsx')
statis = pd.read_excel('../data/통계청 국제표준산업분류 HSCODE 6단위 매핑.xlsx')
customs = pd.read_excel('../data/관세청_HS부호_240101.xlsx')

text_copy = text.copy()
statis_copy = statis.copy()
customs_copy = customs.copy()

print('> 데이터 로드 완료')


# 데이터 전처리

def zero_input(num, x):
    if pd.isna(x):
        return np.nan
    else:
        cnt = num - len(x)
        return '0' * cnt + x
    
def re_sub(x):
    if pd.isna(x):
        return np.nan
    else:
        return re.sub(r'^\((.*?)\)$', r'\1', x)

text_copy['ID'] = text_copy['ID'].astype(str)
text_copy['CODE'] = text_copy['CODE'].astype(str)
text_copy['CODE'] = text_copy['CODE'].apply(lambda x: zero_input(4, x))

statis_copy.columns = [
    'ISIC4_CODE', # ISIC4_국제표준산업분류
    'ISIC4_NAME', # ISIC4_분류명
    'KSIC10_CODE', # KSIC10_한국표준산업분류
    'KSIC10_NAME', # KSIC10_분류명
    'HS2017_CODE', # HS2017_관세통계통합품목분류
    'HS2017_NAME' # HS2017_분류명
]

statis_copy['ISIC4_CODE'] = statis_copy['ISIC4_CODE'].astype(str)
statis_copy['ISIC4_CODE'] = statis_copy['ISIC4_CODE'].replace('nan', np.nan)
statis_copy['ISIC4_CODE'] = statis_copy['ISIC4_CODE'].str.replace('.0', '', regex=False)
statis_copy['ISIC4_CODE'] = statis_copy['ISIC4_CODE'].apply(lambda x: zero_input(4, x))

statis_copy['HS2017_CODE'] = statis_copy['HS2017_CODE'].astype(str)
statis_copy['HS2017_CODE'] = statis_copy['HS2017_CODE'].replace('nan', np.nan)
statis_copy['HS2017_CODE'] = statis_copy['HS2017_CODE'].str.replace('.0', '', regex=False)
statis_copy['HS2017_CODE'] = statis_copy['HS2017_CODE'].apply(lambda x: zero_input(6, x))

customs_copy.columns = [
    'HS_CODE', # HS부호
    'KOR_NAME', # 한글품목명
    'ENG_NAME', # 영문품목명
    'INT_CODE', # 성질통합분류코드
    'INT_NAME' # 성질통합분류명
]

customs_copy['HS_CODE'] = customs_copy['HS_CODE'].astype(str)
customs_copy['HS_CODE'] = customs_copy['HS_CODE'].apply(lambda x: zero_input(10, x))

customs_copy['INT_CODE'] = customs_copy['INT_CODE'].astype(str)
customs_copy['INT_CODE'] = customs_copy['INT_CODE'].replace('nan', np.nan)
customs_copy['INT_CODE'] = customs_copy['INT_CODE'].str.replace('.0', '', regex=False)

customs_copy['INT_NAME'] = customs_copy['INT_NAME'].apply(lambda x: re_sub(x))

text_copy = text_copy.fillna(' ')
statis_copy = statis_copy.fillna(' ')
customs_copy = customs_copy.fillna(' ')

print('> 데이터 전처리 완료')
print('> 데이터 결측치 확인')
print('-----' * 5)
print(text_copy.isnull().sum())
print(statis_copy.isnull().sum())
print(customs_copy.isnull().sum())
print('-----' * 5)


# 데이터 저장 및 로드

text_copy.to_csv('../data/prepro_text.csv', index=False, encoding='utf-8')
statis_copy.to_csv('../data/prepro_statis.csv', index=False, encoding='utf-8')
customs_copy.to_csv('../data/prepro_customs.csv', index=False, encoding='utf-8')

text_prepro = pd.read_csv('../data/prepro_text.csv', dtype=str)
statis_prepro = pd.read_csv('../data/prepro_statis.csv', dtype=str)
customs_prepro = pd.read_csv('../data/prepro_customs.csv', dtype=str)


# csv to jsonl

def csv_to_jsonl(csv_file_path, jsonl_file_path):
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        
        with open(jsonl_file_path, mode='w', encoding='utf-8') as jsonl_file:
            for row in csv_reader:
                jsonl_file.write(json.dumps(row, ensure_ascii=False) + '\n')

csv_to_jsonl('../data/prepro_text.csv', '../data/jsonl_prepro_text.jsonl')
csv_to_jsonl('../data/prepro_statis.csv', '../data/jsonl_prepro_statis.jsonl')
csv_to_jsonl('../data/prepro_customs.csv', '../data/jsonl_prepro_customs.jsonl')
print('> csv to jsonl 완료')

> 데이터 로드 완료
> 데이터 전처리 완료
> 데이터 결측치 확인
-------------------------
ID      0
CODE    0
DSC     0
dtype: int64
ISIC4_CODE     0
ISIC4_NAME     0
KSIC10_CODE    0
KSIC10_NAME    0
HS2017_CODE    0
HS2017_NAME    0
dtype: int64
HS_CODE     0
KOR_NAME    0
ENG_NAME    0
INT_CODE    0
INT_NAME    0
dtype: int64
-------------------------
> csv to jsonl 완료


# Document 구성

> 통계청, 관세청에서 텍스트 스플릿 따로 안 함. 길이가 짧음

## text

In [4]:
file_path = '../data/jsonl_prepro_text.jsonl'

loader = JSONLoader(
    file_path=file_path,
    jq_schema='.',
    text_content=False,
    json_lines=True,
)
temp = loader.load()

seq_num = 1
text_documents = []
for tmp in temp:
    data = json.loads(tmp.page_content)
    doc = Document(
        page_content=data['DSC'], 
        metadata={
            'ID': data['ID'],
            'CODE': data['CODE'],
            'source': '/root/contest-matching-model/data/jsonl_prepro_text.jsonl',
            'seq_num': seq_num,
        }
    )
    text_documents.append(doc)
    seq_num += 1

print(text_documents[0].page_content)
pprint(text_documents[0].metadata)

automotive repair shops, nec  specialized automotive repair, not elsewhere classified, such as fuel service carburetor repair, brake relining, front-end and wheel alignment, and radiator repair. motor vehicle repair and maintenance auto brake lining, installation other automotive mechanical and electrical repair and maintenance maintenance and repair of motor vehicles maintenance and repair of motor vehicles maintenance and repair of motor vehiclesother automotive repair and maintenance
{'CODE': '4520',
 'ID': '1',
 'seq_num': 1,
 'source': '/root/contest-matching-model/data/jsonl_prepro_text.jsonl'}


## statis

In [5]:
file_path = '../data/jsonl_prepro_statis.jsonl'

loader = JSONLoader(
    file_path=file_path,
    jq_schema='.',
    text_content=False,
    json_lines=True,
)
temp = loader.load()

seq_num = 1
statis_documents = []
for tmp in temp:
    data = json.loads(tmp.page_content)
    doc = Document(
        page_content=f"{data['ISIC4_NAME']}\r\n{data['KSIC10_NAME']}\r\n{data['HS2017_NAME']}", # ISIC4, KSIC10, HS2017 순으로 작성됨
        metadata={
            'ISIC4_CODE': data['ISIC4_CODE'],
            'KSIC10_CODE': data['KSIC10_CODE'],
            'HS2017_CODE': data['HS2017_CODE'],
            'source': '/root/contest-matching-model/data/jsonl_prepro_statis.jsonl',
            'seq_num': seq_num,
        }
    )
    statis_documents.append(doc)
    seq_num += 1

print(statis_documents[0].page_content)
pprint(statis_documents[0].metadata)

곡물(쌀 제외), 콩류, 종실유 재배업
종자 및 묘목 생산업
종자
{'HS2017_CODE': '100111',
 'ISIC4_CODE': '0111',
 'KSIC10_CODE': '01123',
 'seq_num': 1,
 'source': '/root/contest-matching-model/data/jsonl_prepro_statis.jsonl'}


## customs

In [6]:
file_path = '../data/jsonl_prepro_customs.jsonl'

loader = JSONLoader(
    file_path=file_path,
    jq_schema='.',
    text_content=False,
    json_lines=True,
)
temp = loader.load()

seq_num = 1
customs_documents = []
for tmp in temp:
    data = json.loads(tmp.page_content)
    doc = Document(
        page_content=f"{data['KOR_NAME']}\r\n{data['ENG_NAME']}\r\n{data['INT_NAME']}", # 한글품목명, 영어품목명, 성질 통합 분류명 순으로 작성됨
        metadata={
            'HS_CODE': data['HS_CODE'],
            'INT_CODE': data['INT_CODE'],
            'source': '/root/contest-matching-model/data/jsonl_prepro_customs.jsonl',
            'seq_num': seq_num,
        }
    )
    customs_documents.append(doc)
    seq_num += 1

print(customs_documents[0].page_content)
pprint(customs_documents[0].metadata)

농가 사육용
For farm breeding
말
{'HS_CODE': '0101211000',
 'INT_CODE': '11020101',
 'seq_num': 1,
 'source': '/root/contest-matching-model/data/jsonl_prepro_customs.jsonl'}


# 벡터스토어 생성

> 통계청, 관세청만 해당함 (텍스트는 인풋 값이어서 벡터스토어에 안 넣음)

In [7]:
# Embedding
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY, 
    model="solar-embedding-1-large"
)

## statis

In [8]:
name = 'statis'
folder_path = f'./faiss_{name}'
if not os.path.exists(folder_path):
    print(f'> {name} Vector Store 생성 중')
    statis_vectorstore = FAISS.from_documents(
        documents=statis_documents,
        embedding=embeddings,
    )
    statis_vectorstore.save_local(folder_path=folder_path)
    print(f'> {name} Vector Store 생성 및 로컬 저장 완료')
else:
    statis_vectorstore = FAISS.load_local(
        folder_path=folder_path, 
        embeddings=embeddings, 
        allow_dangerous_deserialization=True
    )
    print(f'> {name} Vector Store 로컬에서 불러옴')

> statis Vector Store 로컬에서 불러옴


## customs

In [9]:
name = 'customs'
folder_path = f'./faiss_{name}'
if not os.path.exists(folder_path):
    print(f'> {name} Vector Store 생성 중')
    customs_vectorstore = FAISS.from_documents(
        documents=customs_documents,
        embedding=embeddings,
    )
    customs_vectorstore.save_local(folder_path=folder_path)
    print(f'> {name} Vector Store 생성 및 로컬 저장 완료')
else:
    customs_vectorstore = FAISS.load_local(
        folder_path=folder_path, 
        embeddings=embeddings, 
        allow_dangerous_deserialization=True
    )
    print(f'> {name} Vector Store 로컬에서 불러옴')

> customs Vector Store 로컬에서 불러옴


# 적절한 HS CODE 찾는 프로세스

> 텍스트의 jsonl 한 줄 들어옴

> 텍스트의 ISIC4와 통계청의 ISIC4 같은거 찾기 (metadata 끼리 비교) RAG가 중점이니까 텍스트로 하는게 좋을 것 같긴 함<br>유사도 검색 수행도 해서 비교하기

> 조건 거친 통계청의 page_content를 텍스트의 page_content와 비교하여 적절한 통계청 찾기 (점수 기반 유사도 또는 retriever)

> 조건 거친 통계청의 page_content와(단일 또는 복수) 텍스트의 page_content를 컨텍스트로 주고, 관세청의 page_content와 비교

> 관세청 HS_CODE topk(k >= 10) 추출

> 위 과정에서 레퍼런스 잘 챙기기 

## 유사도 검색 수행: 통계청

In [22]:
query = text_documents[0].page_content
print(query)
print(text_documents[0].metadata['CODE'])

automotive repair shops, nec  specialized automotive repair, not elsewhere classified, such as fuel service carburetor repair, brake relining, front-end and wheel alignment, and radiator repair. motor vehicle repair and maintenance auto brake lining, installation other automotive mechanical and electrical repair and maintenance maintenance and repair of motor vehicles maintenance and repair of motor vehicles maintenance and repair of motor vehiclesother automotive repair and maintenance
4520


In [23]:
# vectorstore에서 유사도 검색
statis_similarity = statis_vectorstore.similarity_search(
    query=query,
    k=10
)
i = 1
for doc in statis_similarity:
    print('-----' * 7)
    print(f'{i}.')
    print(doc.page_content)
    print(doc.metadata['ISIC4_CODE'])
    i += 1

-----------------------------------
1.
자동차 정비 및 수리업
자동차 전문 수리업
 
4520
-----------------------------------
2.
자동차를 제외한 운송 장비 수리업
기타 일반 기계 및 장비 수리업
 
3315
-----------------------------------
3.
자동차 정비 및 수리업
자동차 종합 수리업
 
4520
-----------------------------------
4.
자동차 정비 및 수리업
자동차 종합 수리업
 
4520
-----------------------------------
5.
자동차 정비 및 수리업
자동차 세차업
 
4520
-----------------------------------
6.
기계 수리업
기타 일반 기계 및 장비 수리업
 
3312
-----------------------------------
7.
모터사이클 및 관련 부품과 부속품 판매, 정비, 수리업
모터사이클 수리업
 
4540
-----------------------------------
8.
가공 금속 제품 수리업
기타 일반 기계 및 장비 수리업
 
3311
-----------------------------------
9.
컴퓨터 및 주변장비 수리업
기타 일반 기계 및 장비 수리업
 
9511
-----------------------------------
10.
기타 장비 수리업
기타 일반 기계 및 장비 수리업
 
3319


In [24]:
# vectorstore에서 점수에 기반한 유사도 검색
statis_score = statis_vectorstore.similarity_search_with_score(
    query=query,
    k=10
)

i = 1
for doc in statis_score:
    content, score = doc
    print('-----' * 7)
    print(f'{i}.')
    print(content.page_content)
    print(content.metadata['ISIC4_CODE'])
    print(score)
    i += 1

-----------------------------------
1.
자동차 정비 및 수리업
자동차 전문 수리업
 
4520
1.1520796
-----------------------------------
2.
자동차를 제외한 운송 장비 수리업
기타 일반 기계 및 장비 수리업
 
3315
1.1848097
-----------------------------------
3.
자동차 정비 및 수리업
자동차 종합 수리업
 
4520
1.1894037
-----------------------------------
4.
자동차 정비 및 수리업
자동차 종합 수리업
 
4520
1.1894037
-----------------------------------
5.
자동차 정비 및 수리업
자동차 세차업
 
4520
1.2517118
-----------------------------------
6.
기계 수리업
기타 일반 기계 및 장비 수리업
 
3312
1.2894791
-----------------------------------
7.
모터사이클 및 관련 부품과 부속품 판매, 정비, 수리업
모터사이클 수리업
 
4540
1.2920146
-----------------------------------
8.
가공 금속 제품 수리업
기타 일반 기계 및 장비 수리업
 
3311
1.2933953
-----------------------------------
9.
컴퓨터 및 주변장비 수리업
기타 일반 기계 및 장비 수리업
 
9511
1.3055391
-----------------------------------
10.
기타 장비 수리업
기타 일반 기계 및 장비 수리업
 
3319
1.315347


## 유사도 검색 수행: 관세청

In [14]:
query = text_documents[0].page_content
print(query)

automotive repair shops, nec  specialized automotive repair, not elsewhere classified, such as fuel service carburetor repair, brake relining, front-end and wheel alignment, and radiator repair. motor vehicle repair and maintenance auto brake lining, installation other automotive mechanical and electrical repair and maintenance maintenance and repair of motor vehicles maintenance and repair of motor vehicles maintenance and repair of motor vehiclesother automotive repair and maintenance


In [16]:
# vectorstore에서 유사도 검색
customs_similarity = customs_vectorstore.similarity_search(
    query=query,
    k=10
)
i = 1
for doc in customs_similarity:
    print('-----' * 7)
    print(f'{i}.')
    print(doc.page_content)
    print(doc.metadata['HS_CODE'])
    i += 1

-----------------------------------
1.
연료나 윤활유 급유용 펌프(주유소나 정비소에서 사용하는 형태로 한정한다)
Pumps for dispensing fuel or lubricants, of the type used in filling-stations or in garages
액체펌프
8413110000
-----------------------------------
2.
금속세정액, 유압액, 브레이크액, 부동액 폐기물
Wastes of metal pickling liquors, hydraulic fluids, brake fluids and anti-freeze fluids
기타 화학공업제품
3825500000
-----------------------------------
3.
비구동 차축과 그 부분품
Non-driving axles and parts thereof
자동차 부품
8708502000
-----------------------------------
4.
기타
Other
자동차 부품
8708959000
-----------------------------------
5.
기타
Other
자동차 부품
8708999000
-----------------------------------
6.
기타
Other
자동차 부품
8716909000
-----------------------------------
7.
기타
Other
자동차 부품
8708290000
-----------------------------------
8.
기타
Other
자동차 부품
8511109000
-----------------------------------
9.
기타
Other
자동차 부품
8511209000
-----------------------------------
10.
기타
Other
자동차 부품
8511309000


In [19]:
query

'automotive repair shops, nec  specialized automotive repair, not elsewhere classified, such as fuel service carburetor repair, brake relining, front-end and wheel alignment, and radiator repair. motor vehicle repair and maintenance auto brake lining, installation other automotive mechanical and electrical repair and maintenance maintenance and repair of motor vehicles maintenance and repair of motor vehicles maintenance and repair of motor vehiclesother automotive repair and maintenance'

In [17]:
# vectorstore에서 점수에 기반한 유사도 검색
customs_score = customs_vectorstore.similarity_search_with_score(
    query=query,
    k=10
)
i = 1
for doc in customs_score:
    content, score = doc
    print('-----' * 7)
    print(f'{i}.')
    print(content.page_content)
    print(content.metadata['HS_CODE'])
    print(score)
    i += 1

-----------------------------------
1.
연료나 윤활유 급유용 펌프(주유소나 정비소에서 사용하는 형태로 한정한다)
Pumps for dispensing fuel or lubricants, of the type used in filling-stations or in garages
액체펌프
8413110000
1.2886624
-----------------------------------
2.
금속세정액, 유압액, 브레이크액, 부동액 폐기물
Wastes of metal pickling liquors, hydraulic fluids, brake fluids and anti-freeze fluids
기타 화학공업제품
3825500000
1.3514187
-----------------------------------
3.
비구동 차축과 그 부분품
Non-driving axles and parts thereof
자동차 부품
8708502000
1.3609972
-----------------------------------
4.
기타
Other
자동차 부품
8708959000
1.36557
-----------------------------------
5.
기타
Other
자동차 부품
8708999000
1.36557
-----------------------------------
6.
기타
Other
자동차 부품
8716909000
1.36557
-----------------------------------
7.
기타
Other
자동차 부품
8708290000
1.3657708
-----------------------------------
8.
기타
Other
자동차 부품
8511109000
1.3659075
-----------------------------------
9.
기타
Other
자동차 부품
8511209000
1.3659075
-----------------------------------
10.
기타
Other
자

# LangSmith hub

In [21]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
prompt = hub.pull("rlm/rag-prompt")

```
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Question: {question}
Context: {context}
Answer:
```